In [ ]:
import os, sys
import pandas as pd
import numpy as np
import lisa.algorithms
import colorsys
import shutil
from IPython.core.display import display, HTML, display_html, Markdown
from lisa.platforms import supported_platforms
from lisa.usecases import get_elemental_timeline
from lisa.automation import automation
from lisa.algorithms import time_series_analysis as ts
from pathlib import Path
from lisa import trace
import warnings
warnings.filterwarnings('ignore')

use_bokeh = eval(os.environ.get('PTAS_LISA_USE_BOKEH', 'True'))

if use_bokeh:
    from bokeh.models import HoverTool, BoxZoomTool, ResetTool, PanTool, SaveTool, RedoTool, UndoTool, LogColorMapper, LinearAxis, Range1d
    from bokeh.plotting import figure, ColumnDataSource, gridplot
    from bokeh.io import show, reset_output, output_notebook
    from bokeh.resources import INLINE
    from bokeh.layouts import row
    from bokeh.palettes import Greys256
    from bokeh.transform import dodge

def clear_output(wait=False):
    from IPython.core.interactiveshell import InteractiveShell
    if InteractiveShell.initialized():
        InteractiveShell.instance().display_pub.clear_output(wait)
    else:
        print('\033[2K\r', end='')
        sys.stdout.flush()
        print('\033[2K\r', end='')
        sys.stderr.flush()

clear_output()
reset_output()
output_notebook(resources=INLINE, hide_banner=True)

display(HTML("<style>.container { width:100% !important; }</style>"))

In [61]:
#Notebook assumes Certified_Ftrace folder structure as input. Each usecase should have only 1 ftrace file

knp_path =  r"C:\KNP_ExRedata\Ftrace\Ftrace\KNP"
pakala_path = r"C:\KNP_ExRedata\Ftrace\Ftrace\PKL"

# Pakala network path = \\pttla-5\d_drive\Dropbox\Surya\Pakala\WL_Cache\LISA_Cache_Pakala_M923_CS
# Lanai network path = \\pttla-5\d_drive\Dropbox\Surya\Pakala\WL_Cache\LISA_Cache_Lanai

In [30]:
import bz2

i = 0
for path in Path(knp_path).rglob('*_ftrace.txt*'):
    if(path.name.endswith('.bz2')):
        extract_to_dir = path.parent
        decom_file_name = path.stem
        i = i +1
        decom_path = extract_to_dir / decom_file_name
        if not os.path.exists(decom_path):
            with bz2.open(path, 'rb') as f_in:
                with open(decom_path, 'wb') as f_out:
                    shutil.copyfileobj(f_in,f_out)
                    print(f"Decompress Yes! '{i, decom_path}'")

Decompress Yes! '(4, WindowsPath('C:/KNP_ExRedata/Ftrace/Ftrace/KNP/SRVKGH02/3/1/DebugLogs/CPULogs/SRVKGH02_ftrace.txt'))'
Decompress Yes! '(5, WindowsPath('C:/KNP_ExRedata/Ftrace/Ftrace/KNP/WW/1/1/DebugLogs/CPULogs/WW_ftrace.txt'))'


In [33]:
# import bz2

# i = 0
# for path in Path(pakala_path).rglob('*_ftrace.txt*'):
#     if(path.name.endswith('.bz2')):
#         extract_to_dir = path.parent
#         decom_file_name = path.stem
#         i = i +1
#         decom_path = extract_to_dir / decom_file_name
#         if not os.path.exists(decom_path):
#             with bz2.open(path, 'rb') as f_in:
#                 with open(decom_path, 'wb') as f_out:
#                     shutil.copyfileobj(f_in,f_out)
#                     print(f"Decompress Yes! '{i, decom_path}'")

In [63]:
from lisa import trace
def find_addtl_files(trace_path):
    addtl_file_keywords = ["ps.log", "gfx","interconnect_summary.txt","background_tasks.txt",
                       "topapp_tasks.txt","foreground_tasks.txt", "profile.report",
                       "ddr_stats_start.txt", "ddr_stats_end.txt",
                       "cpucp_log.txt", "cpucp_log_offset.txt"]
    addtl_files = {}
    trace_dir = os.path.dirname(trace_path)
    for filename in os.listdir(trace_dir):
        file_path = os.path.join(trace_dir, filename)
        for keyword in addtl_file_keywords:
            if keyword in filename:
                if keyword not in addtl_files.keys():
                    addtl_files[keyword] = [file_path]
                else:
                    addtl_files[keyword].append(file_path)
                break
    return addtl_files


trace_list_pakala=[]
trace_list_knp=[]
trace_list = []
i = 0
for path in Path(knp_path).rglob('*_ftrace.txt'):
    trace_plat = supported_platforms['platform_SM8850']
    trace_path = str(path)
    trace_alias = trace_path.split('\\')[-6]
    print(trace_alias, i, trace_path)     
    trace_addtl_files = find_addtl_files(trace_path)

    t = trace.Trace(data_dir=trace_path,
                    extra_files=trace_addtl_files,
                    plat_info=trace_plat,
                    events=trace_plat["trace events"],
                    name=trace_alias,
                    validate_cache=True)
    trace_list_knp.append(t)
    trace_list.append(t)
    i=i+1
    print("End")
       

for path in Path(pakala_path).rglob('*_ftrace.txt'):
    trace_plat = supported_platforms['platform_SM8750']
    trace_path = str(path)
    trace_alias = trace_path.split('\\')[-6] 
    print(trace_alias, i, trace_path)
    trace_addtl_files = find_addtl_files(trace_path)

    t = trace.Trace(data_dir=trace_path,
                    extra_files=trace_addtl_files,
                    plat_info=trace_plat,
                    events=trace_plat["trace events"],
                    name=trace_alias,
                    validate_cache=True)
    trace_list_pakala.append(t)
    trace_list.append(t)
    i=i+1
    print("End")

GPHDR120 0 C:\KNP_ExRedata\Ftrace\Ftrace\KNP\GPHDR120\1\1\Debuglogs\CPULogs\GPHDR120_ftrace.txt
[Errno 2] No such file or directory: 'c:\\lisa_v2\\lisa\\platforms\\clock_plan\\SM8850.xlsx'
[Errno 2] No such file or directory: 'c:\\lisa_v2\\lisa\\platforms\\leakage_data\\SM8850.xlsx'
 Loading event from cache : cpu_frequency_devlib                                               
End
HOK02W 1 C:\KNP_ExRedata\Ftrace\Ftrace\KNP\HOK02W\1\1\DebugLogs\CPULogs\HOK02W_ftrace.txt
[Errno 2] No such file or directory: 'c:\\lisa_v2\\lisa\\platforms\\clock_plan\\SM8850.xlsx'
[Errno 2] No such file or directory: 'c:\\lisa_v2\\lisa\\platforms\\leakage_data\\SM8850.xlsx'
 Loading event from cache : cpu_frequency_devlib                                               
End
SRVKGH02 2 C:\KNP_ExRedata\Ftrace\Ftrace\KNP\SRVKGH02\3\1\DebugLogs\CPULogs\SRVKGH02_ftrace.txt
[Errno 2] No such file or directory: 'c:\\lisa_v2\\lisa\\platforms\\clock_plan\\SM8850.xlsx'
[Errno 2] No such file or directory: 'c:\\lisa_v2

In [64]:
trace_list_pakala_beifen = trace_list_pakala
trace_list_knp_beifen = trace_list_knp

for i, trace in enumerate(trace_list_pakala):
    print(f"--- Trace {i} ---")
    print(f"Name: {trace.name}")



--- Trace 0 ---
Name: GPHDR120
--- Trace 1 ---
Name: HOK02W
--- Trace 2 ---
Name: SRVKGH02
--- Trace 3 ---
Name: WWVKCP01
--- Trace 4 ---
Name: ZZZVKLS01


In [65]:

for i, trace in enumerate(trace_list_knp):
    print(f"--- Trace {i} ---")
    print(f"Name: {trace.name}")



--- Trace 0 ---
Name: GPHDR120
--- Trace 1 ---
Name: HOK02W
--- Trace 2 ---
Name: SRVKGH02
--- Trace 3 ---
Name: WWVKCP01
--- Trace 4 ---
Name: ZZZVKLS01


In [66]:
builds_dict = {'KNP': trace_list_knp,'Pakala': trace_list_pakala}

In [67]:
usecasestestplans_dict = {
    "SSDoU_V8": {
        "SyncAndIdle": {
            "weightage": 43.57,
            "aliases": [
                "IdleMode",
                "SYNC_MR2",
                "SyncAndIdle",
                "BackgroundSync"
            ]
        },
        "SnI-MessageReceive (4G)": {
            "weightage": 10.04,
            "aliases": [
                "MsgRx4G",
                "SMSCheck",
                "SnI-MessageReceive (4G)",
                "SYNC_MR2"
            ]
        },
        "SnI-DisplayOnOff (APM)": {
            "weightage": 3.01,
            "aliases": [
                "SONOFF03A",
                "SnI-DisplayOnOff (APM)"
            ]
        },
        "LCDOnIdle (APM)": {
            "weightage": 0.0,
            "aliases": [
                "LCD05A",
                "LCDOnIdle",
                "LCDOnIdle (APM)"
            ]
        },
        "MenuListFM (APM)": {
            "weightage": 2.17,
            "aliases": [
                "MLF02A",
                "MenuListFM (APM)"
            ]
        },
        "AppLaunchBrowser (WiFi)": {
            "weightage": 2.17,
            "aliases": [
                "AppLaunchBrowser (WiFi)",
                "BrowserOpen",
                "LWB03W",
                "LaunchChrome"
            ]
        },
        "Angry GPS (APM)": {
            "weightage": 2.17,
            "aliases": [
                "ANGPS01A",
                "Angry GPS (APM)"
            ]
        },
        "VoLTE (4G)": {
            "weightage": 2.6,
            "aliases": [
                "LTE",
                "VoLTE (4G)"
            ]
        },
        "YoutubeMusicStreaming (WiFi)": {
            "weightage": 1.3,
            "aliases": [
                "AS02W",
                "YoutubeMusicStreaming (WiFi)"
            ]
        },
        "YoutubeVideoStreaming (WiFi)": {
            "weightage": 6.51,
            "aliases": [
                "VS06W",
                "YoutubeVideoStreaming (WiFi)"
            ]
        },
        "Click to Search (WiFi)": {
            "weightage": 2.6,
            "aliases": [
                "CLICKS01W",
                "Click to Search (WiFi)"
            ]
        },
        "Google Chat (5G)": {
            "weightage": 5.56,
            "aliases": [
                "GCHAT01-5G",
                "Google Chat (5G)"
            ]
        },
        "Internet Translation (WiFi)": {
            "weightage": 3.11,
            "aliases": [
                "INTRANS01W",
                "Internet Translation (WiFi)"
            ]
        },
        "Game Simulator (APM)": {
            "weightage": 2.6,
            "aliases": [
                "GSIMU01A",
                "Game Simulator (APM)"
            ]
        },
        "BrawlStars (WiFi)": {
            "weightage": 2.17,
            "aliases": [
                "BRS01W",
                "BrawlStars (WiFi)"
            ]
        },
        "ClashRoyale (5G)": {
            "weightage": 1.73,
            "aliases": [
                "CLR03-5G",
                "ClashRoyale (5G)"
            ]
        },
        "Sbrowser (WiFi)": {
            "weightage": 3.47,
            "aliases": [
                "Sbrowser (WiFi)",
                "WB",
                "WB09W"
            ]
        },
        "BrowserPageLoading (4G)": {
            "weightage": 3.47,
            "aliases": [
                "BrowserPageLoading (4G)",
                "WBPL01"
            ]
        }
    },
    "GDoU_5.2": {
        "SBYBG02W": {
            "weightage": 26.6,
            "aliases": [
                "SBYBG02W"
            ]
        },
        "RMSG01-5G": {
            "weightage": 3.0,
            "aliases": [
                "RMSG01-5G"
            ]
        },
        "QCE20A": {
            "weightage": 0.6,
            "aliases": [
                "QCE20A"
            ]
        },
        "DOUYIN02-5G": {
            "weightage": 1.0,
            "aliases": [
                "DOUYIN02-5G"
            ]
        },
        "AU45A": {
            "weightage": 2.5,
            "aliases": [
                "AU45A"
            ]
        },
        "VS12W": {
            "weightage": 3.0,
            "aliases": [
                "VS12W"
            ]
        },
        "VS11-5G": {
            "weightage": 3.0,
            "aliases": [
                "VS11-5G"
            ]
        },
        "VS13W": {
            "weightage": 2.0,
            "aliases": [
                "VS13W"
            ]
        },
        "DOUYIN01W": {
            "weightage": 9.0,
            "aliases": [
                "DOUYIN01W"
            ]
        },
        "GP01W": {
            "weightage": 2.0,
            "aliases": [
                "GP01W"
            ]
        },
        "HOK01": {
            "weightage": 3.0,
            "aliases": [
                "HOK01"
            ]
        },
        "QQS01-5G": {
            "weightage": 1.0,
            "aliases": [
                "QQS01-5G"
            ]
        },
        "FORTNITEW": {
            "weightage": 1.5,
            "aliases": [
                "FORTNITEW"
            ]
        },
        "GCB02-5G": {
            "weightage": 4.0,
            "aliases": [
                "GCB02-5G"
            ]
        },
        "TNEWS01W": {
            "weightage": 2.5,
            "aliases": [
                "TNEWS01W"
            ]
        },
        "FB05W": {
            "weightage": 4.0,
            "aliases": [
                "FB05W"
            ]
        },
        "WCHT02W": {
            "weightage": 7.0,
            "aliases": [
                "WCHT02W"
            ]
        },
        "PMWCHT02W": {
            "weightage": 4.0,
            "aliases": [
                "PMWCHT02W"
            ]
        },
        "WHA04W": {
            "weightage": 3.0,
            "aliases": [
                "WHA04W"
            ]
        },
        "WB01-5G": {
            "weightage": 2.5,
            "aliases": [
                "WB01-5G"
            ]
        },
        "VOWCHT01-5G": {
            "weightage": 1.5,
            "aliases": [
                "VOWCHT01-5G"
            ]
        },
        "VIWCHT01W": {
            "weightage": 3.0,
            "aliases": [
                "VIWCHT01W"
            ]
        },
        "VOLTE6-D4": {
            "weightage": 2.3,
            "aliases": [
                "VOLTE6-D4"
            ]
        },
        "MSR04A": {
            "weightage": 2.0,
            "aliases": [
                "MSR04A"
            ]
        },
        "GPS09": {
            "weightage": 1.0,
            "aliases": [
                "GPS09"
            ]
        },
        "APPLNCH03A": {
            "weightage": 3.0,
            "aliases": [
                "APPLNCH03A"
            ]
        },
        "APPLNCH04A-G": {
            "weightage": 2.0,
            "aliases": [
                "APPLNCH04A-G"
            ]
        }
    },
    "GameDB": {
        "GSVKFFC02": {
            "weightage": 16.7,
            "aliases": [
                "GSVKFFC02"
            ]
        },
        "GPHDR120": {
            "weightage": 16.7,
            "aliases": [
                "GPHDR120"
            ]
        },
        "HOK02W": {
            "weightage": 16.7,
            "aliases": [
                "HOK02W"
            ]
        },
        "SRVKGH02": {
            "weightage": 16.7,
            "aliases": [
                "SRVKGH02"
            ]
        },
        "WWVKCP01": {
            "weightage": 16.7,
            "aliases": [
                "WWVKCP01"
            ]
        },
        "ZZZVKLS01": {
            "weightage": 16.7,
            "aliases": [
                "ZZZVKLS01"
            ]
        }
    }
}

In [68]:
KPI_list = ['duration',
'line_cnt',
'commit_cnt',
'render_t',
'fps',
'insts_no_swapper_tot',
'cycles_no_swapper_tot',
'ipc_no_swapper_tot',
'insts_tot',
'cycles_tot',
'ipc_tot',
'insts_s',
'wl_pct_s',
'cycles_s',
'ipc_s',
'insts_g',
'wl_pct_g',
'cycles_g',
'ipc_g',
'insts_t',
'wl_pct_t',
'cycles_t',
'ipc_t',
'insts_p',
'wl_pct_p',
'cycles_p',
'ipc_p',
'l2_misses_no_swapper_tot',
'l2_misses_no_swapper_pki_tot',
'l2_misses_no_swapper_s',
'l2_misses_pki_no_swapper_s',
'l2_misses_no_swapper_t',
'l2_misses_pki_no_swapper_t',
'l2_misses_no_swapper_g',
'l2_misses_pki_no_swapper_g',
'l2_misses_no_swapper_p',
'l2_misses_pki_no_swapper_p',
'l2_misses_tot',
'l2_misses_pki_tot',
'l2_misses_s',
'l2_misses_pki_s',
'l2_misses_t',
'l2_misses_pki_t',
'l2_misses_g',
'l2_misses_pki_g',
'l2_misses_p',
'l2_misses_pki_p',
'l3_misses_no_swapper_tot',
'l3_misses_no_swapper_pki_tot',
'l3_misses_no_swapper_s',
'l3_misses_pki_no_swapper_s',
'l3_misses_no_swapper_t',
'l3_misses_pki_no_swapper_t',
'l3_misses_no_swapper_g',
'l3_misses_pki_no_swapper_g',
'l3_misses_no_swapper_p',
'l3_misses_pki_no_swapper_p',
'l3_misses_tot',
'l3_misses_pki_tot',
'l3_misses_s',
'l3_misses_pki_s',
'l3_misses_t',
'l3_misses_pki_t',
'l3_misses_g',
'l3_misses_pki_g',
'l3_misses_p',
'l3_misses_pki_p',
'fe_stalls_no_swapper_tot',
'fe_stalls_no_swapper_pki_tot',
'fe_stalls_no_swapper_s',
'fe_stalls_pki_no_swapper_s',
'fe_stalls_no_swapper_t',
'fe_stalls_pki_no_swapper_t',
'fe_stalls_no_swapper_g',
'fe_stalls_pki_no_swapper_g',
'fe_stalls_no_swapper_p',
'fe_stalls_pki_no_swapper_p',
'fe_stalls_tot',
'fe_stalls_pki_tot',
'fe_stalls_s',
'fe_stalls_pki_s',
'fe_stalls_t',
'fe_stalls_pki_t',
'fe_stalls_g',
'fe_stalls_pki_g',
'fe_stalls_p',
'fe_stalls_pki_p',
'llc_misses_tot',
'llc_misses_pki_tot',
'llc_misses_s',
'llc_misses_pki_s',
'llc_misses_g',
'llc_misses_pki_g',
'llc_misses_p',
'llc_misses_pki_p',
'top5_insts',
'droidbg_insts',
'ddr_mbps',
'llcc_gold_mbps',
'llcc_prime_mbps',
'llcc_mbps',
'mig_intra_tot',
'mig_inter_gs',
'mig_inter_st',
'mig_inter_ps',
'mig_inter_gt',
'mig_inter_gp',
'mig_inter_pt',
'mig_intra_pkw',
'mig_inter_pkw_gs',
'mig_inter_pkw_st',
'mig_inter_pkw_ps',
'mig_inter_pkw_gt',
'mig_inter_pkw_gp',
'mig_inter_pkw_pt',
'preempt_cnt',
'preempt_pkw',
'wake_pc_tot_cnt',
'wake_pc_s_cnt',
'wake_pc_gp_cnt',
'wake_wfi_tot_cnt',
'wake_tot_cnt',
'freq_avg_s',
'freq_avg_g',
'freq_avg_t',
'freq_avg_p',
'l3_collapse_cnt',
'l3_collapse_dur',
'coloc_cnt',
'coloc_t',
'fboost_t',
'cboost_t',
'rboost_t',
'tot_boost_t',
'tot_boost_cnt',
'state1_time_residency',
'state_transition_cnt',
'cluster_enter_cnt',
'cluster_exit_cnt',
'cl5_time_tot']


In [69]:
def get_usecase_name(alias: str, usecases_by_plan: dict) -> str:
    """
    Given an alias, return the corresponding usecase name by searching the original dictionary.
    If multiple matches exist, returns the first one found.
    If no match, returns None.
    """
    alias_lower = alias.strip().casefold()
    for plan, usecases in usecases_by_plan.items():
        for usecase_name, details in usecases.items():
            # Check canonical name
            if usecase_name.casefold() == alias_lower:
                return usecase_name
            # Check aliases
            if any(a.casefold() == alias_lower for a in details.get("aliases", [])):
                return usecase_name
    return None

In [70]:
def print_dict_table(dictionary):
    for key, value in dictionary.items():
        display(Markdown("## {}".format(key)))
        display(HTML(value.style.render()))
    pass

def print_dict_table_with_pallete(dictionary):
    for key, value in dictionary.items():
        display(Markdown("## {}".format(key)))
        styled_df = value.style.background_gradient(cmap='Pastel2', axis = 1)
        display(HTML(styled_df.render()))
    pass

In [71]:
def process_trace_list(build, trace_list, usecasestestplans_dict, KPI_list):
    # Compare KPIs
    kpi_df = trace_list[0].analysis.general_kpi.compare_general_kpis_with(trace_list[1:]).iloc[1:].T       
    
    # Clean and map index
    kpi_df.index = pd.Index([st.replace("value ", "") for st in kpi_df.index], name=build)
    kpi_df.index = kpi_df.index.astype(str).map(lambda x: get_usecase_name(x, usecasestestplans_dict))
    print(kpi_df.index)

    # Reindex and format
    kpi_df = kpi_df.reindex(index=usecasestestplans_dict['GameDB'].keys(), fill_value=0)
    return kpi_df.astype(float).round(1)

def compute_deltas(output_dict):
#     print(output_dict['KNP'])
    kpi_order = output_dict['KNP'].columns.tolist()

#     print(output_dict['KNP - Pakala'].reindex(columns=kpi_order, fill_value=0).columns)
    output_dict['KNP - Pakala'] = (output_dict['KNP'] - output_dict['Pakala']).reindex(columns=kpi_order, fill_value=0)
    output_dict['% Delta'] = (output_dict['KNP - Pakala'] / output_dict['Pakala']).reindex(columns=kpi_order, fill_value=0) * 100
    output_dict['% Delta'] = output_dict['% Delta'].round(2)
    return output_dict

def export_to_excel(output_dict, filename='General_KPIs.xlsx'):
    with pd.ExcelWriter(filename) as writer:
        for build, output in output_dict.items():
            output.to_excel(writer, sheet_name=build, index=True)

def generate_kpi_report(builds_dict, usecasestestplans_dict, KPI_list):
    output_dict = {}
    for build, trace_list in builds_dict.items():
        output_dict[build] = process_trace_list(build, trace_list, usecasestestplans_dict, KPI_list)

    output_dict = compute_deltas(output_dict)

    display(Markdown("# General KPIs"))
    print_dict_table(output_dict)
    export_to_excel(output_dict)

    return output_dict

In [70]:
output_dict = generate_kpi_report(builds_dict, usecasestestplans_dict, KPI_list)

Events [halt_cpus] not found, cannot get halted cpus.
Event [cluster_enter] not found, plot DISABLED!
Event [cluster_exit] not found, plot DISABLED!
Event [cluster_pred_hist] not found, plot DISABLED!
Events [halt_cpus] not found, cannot get halted cpus.
Event [cluster_enter] not found, plot DISABLED!
Event [cluster_exit] not found, plot DISABLED!
Event [cluster_pred_hist] not found, plot DISABLED!
Events [halt_cpus] not found, cannot get halted cpus.
Event [cluster_enter] not found, plot DISABLED!
Event [cluster_exit] not found, plot DISABLED!
Event [cluster_pred_hist] not found, plot DISABLED!
Events [halt_cpus] not found, cannot get halted cpus.
Event [cluster_enter] not found, plot DISABLED!
Event [cluster_exit] not found, plot DISABLED!
Event [cluster_pred_hist] not found, plot DISABLED!
Events [halt_cpus] not found, cannot get halted cpus.
Event [cluster_enter] not found, plot DISABLED!
Event [cluster_exit] not found, plot DISABLED!
Event [cluster_pred_hist] not found, plot DISAB

Index(['GPHDR120', 'HOK02W', 'SRVKGH02', 'WWVKCP01', 'ZZZVKLS01'], dtype='object', name='KNP')


Events [halt_cpus] not found, cannot get halted cpus.
Event [cluster_enter] not found, plot DISABLED!
Event [cluster_exit] not found, plot DISABLED!
Event [cluster_pred_hist] not found, plot DISABLED!
Events [halt_cpus] not found, cannot get halted cpus.
Event [cluster_enter] not found, plot DISABLED!
Event [cluster_exit] not found, plot DISABLED!
Event [cluster_pred_hist] not found, plot DISABLED!
Events [halt_cpus] not found, cannot get halted cpus.
Event [cluster_enter] not found, plot DISABLED!
Event [cluster_exit] not found, plot DISABLED!
Event [cluster_pred_hist] not found, plot DISABLED!
Events [halt_cpus] not found, cannot get halted cpus.
Event [cluster_enter] not found, plot DISABLED!
Event [cluster_exit] not found, plot DISABLED!
Event [cluster_pred_hist] not found, plot DISABLED!
Event [cluster_enter] not found, plot DISABLED!
Event [cluster_exit] not found, plot DISABLED!
Event [cluster_pred_hist] not found, plot DISABLED!
Events [halt_cpus] not found, cannot get halted c

Index(['GPHDR120', 'GSVKFFC02', 'HOK02W', 'SRVKGH02', 'WWVKCP01', 'ZZZVKLS01'], dtype='object', name='Pakala')


# General KPIs

...

kpi,duration,line_cnt,commit_cnt,render_t,fps,insts_no_swapper_tot,cycles_no_swapper_tot,ipc_no_swapper_tot,insts_tot,cycles_tot,ipc_tot,insts_g,wl_pct_g,cycles_g,ipc_g,insts_p,wl_pct_p,cycles_p,ipc_p,l2_misses_no_swapper_tot,l2_misses_no_swapper_pki_tot,l2_misses_no_swapper_g,l2_misses_pki_no_swapper_g,l2_misses_no_swapper_p,l2_misses_pki_no_swapper_p,l2_misses_tot,l2_misses_pki_tot,l2_misses_g,l2_misses_pki_g,l2_misses_p,l2_misses_pki_p,l3_misses_no_swapper_tot,l3_misses_no_swapper_pki_tot,l3_misses_no_swapper_g,l3_misses_pki_no_swapper_g,l3_misses_no_swapper_p,l3_misses_pki_no_swapper_p,l3_misses_tot,l3_misses_pki_tot,l3_misses_g,l3_misses_pki_g,l3_misses_p,l3_misses_pki_p,fe_stalls_no_swapper_tot,fe_stalls_no_swapper_pki_tot,fe_stalls_no_swapper_g,fe_stalls_pki_no_swapper_g,fe_stalls_no_swapper_p,fe_stalls_pki_no_swapper_p,fe_stalls_tot,fe_stalls_pki_tot,fe_stalls_g,fe_stalls_pki_g,fe_stalls_p,fe_stalls_pki_p,llc_misses_tot,llc_misses_pki_tot,llc_misses_g,llc_misses_pki_g,llc_misses_p,llc_misses_pki_p,top5_insts,droidbg_insts,ddr_mbps,llcc_gold_mbps,llcc_prime_mbps,mig_intra_tot,mig_inter_gp,mig_intra_pkw,mig_inter_pkw_gp,preempt_cnt,preempt_pkw,wake_pc_tot_cnt,wake_pc_s_cnt,wake_pc_gp_cnt,wake_wfi_tot_cnt,wake_tot_cnt,freq_avg_g,freq_avg_p,l3_collapse_cnt,l3_collapse_dur,coloc_cnt,coloc_t,fboost_t,cboost_t,rboost_t,tot_boost_t,tot_boost_cnt,state1_time_residency,state_transition_cnt,active_tot,insts_tot_by_active_tot
Pakala,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
GSVKFFC02,23.5,9.22271e+06,1420,23.5,60.4,116438,135852,0.9,126923,163283,0.8,99491.4,78.4,145144,0.7,27431.9,21.6,18138.9,1.5,470.4,4,388,4.3,82.4,3.1,479.2,3.8,395.1,4,84.1,3.1,0,0,0,0,0,0,0,0,0,0,0,0,41390.8,355.5,34536.6,385.2,6854.2,255.9,51000.8,401.8,43701.2,439.2,7299.6,266.1,2613.7,20.6,2008.7,20.2,604.9,22.1,76637,0.4,0,15636.2,5154.5,107169,2806,197.1,5.2,23147,42.6,7985,0,7985,535751,543735,2257.1,1106.7,0,0,0,0,0,0,0,0,0,100,0,80.2,1583.6
GPHDR120,60,1.72799e+07,7157,60,119.3,171349,198718,0.9,189723,237891,0.8,156075,82.3,210797,0.7,33648.2,17.7,27094.2,1.2,500.9,2.9,445.8,3.2,55.2,1.7,513.3,2.7,456.1,2.9,57.1,1.7,0,0,0,0,0,0,0,0,0,0,0,0,79799,465.7,68993.3,497.8,10805.7,330,93295,491.7,81783.3,524,11511.8,342.1,3621.5,19.1,3124.5,20,497,14.8,119184,0.1,0,10048.9,1963.1,435679,6872,552.8,8.7,73243,92.9,18094,0,18094,770015,788111,1588.9,1309.7,0,0,0,0,0,0,0,0,0,100,0,151.2,1254.7
HOK02W,57.8,1.51454e+07,6957,57.8,120.4,228167,179081,1.3,241770,205698,1.2,174772,72.3,163359,1.1,66998.1,27.7,42339.7,1.6,550.9,2.4,426.4,2.6,124.5,1.9,564,2.3,437.4,2.5,126.6,1.9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5253.3,21.7,3956,22.6,1297.3,19.4,190455,nan,0,13105.2,4668.8,144166,8542,221.4,13.1,40273,61.8,22308,0,22308,628884,651191,1473.6,1460,0,0,0,0,0,0,0,0,0,100,0,138.9,1740.2
SRVKGH02,57.8,1.24384e+07,3446,57.8,59.6,314400,237553,1.3,327070,258962,1.3,142335,43.5,108703,1.3,184736,56.5,150258,1.2,1267.5,4,277.6,2.1,989.9,5.4,1283.3,3.9,287.6,2,995.7,5.4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,7944.3,24.3,2282.1,16,5662.2,30.6,233802,nan,0,8683.5,17201.2,212489,3115,342.4,5,79016,127.3,45214,0,45214,575396,620611,808,1898.9,0,0,0,0,0,0,0,0,0,100,0,212.1,1542.3
WWVKCP01,46.1,1.022e+07,2734,46.1,59.3,173072,183998,0.9,183744,207617,0.9,150739,82,183880,0.8,33005.5,18,23736.7,1.4,630.9,3.6,533.3,3.8,97.6,3,640,3.5,541.2,3.6,98.8,3,0,0,0,0,0,0,0,0,0,0,0,0,64112.8,370.4,54841.2,389.6,9271.6,287,72047,392.1,62361.3,413.7,9685.7,293.5,3994.3,21.7,2984,19.8,1010.3,30.6,111704,nan,0,12452.6,5045.1,159807,5045,312,9.8,38200,74.6,34344,0,34344,477888,512231,1812.7,1248.6,0,0,0,0,0,0,0,0,0,0.1,2,118.3,1553
ZZZVKLS01,59.1,2.42497e+07,3565,59.1,60.3,358742,297640,1.2,386005,356301,1.1,296372,76.8,282526,1,89633.5,23.2,73774.7,1.2,914.1,2.5,597.5,2.2,316.5,3.6,937.3,2.4,616,2.1,321.3,3.6,0,0,0,0,0,0,0,0,0,0,0,0,93751.2,261.3,62752.2,231.5,30999,353.8,111982,290.1,79052.9,266.7,32929,367.4,6616.2,17.1,4377.5

## KNP - Pakala

kpi,duration,line_cnt,commit_cnt,render_t,fps,insts_no_swapper_tot,cycles_no_swapper_tot,ipc_no_swapper_tot,insts_tot,cycles_tot,ipc_tot,insts_g,wl_pct_g,cycles_g,ipc_g,insts_p,wl_pct_p,cycles_p,ipc_p,l2_misses_no_swapper_tot,l2_misses_no_swapper_pki_tot,l2_misses_no_swapper_g,l2_misses_pki_no_swapper_g,l2_misses_no_swapper_p,l2_misses_pki_no_swapper_p,l2_misses_tot,l2_misses_pki_tot,l2_misses_g,l2_misses_pki_g,l2_misses_p,l2_misses_pki_p,l3_misses_no_swapper_tot,l3_misses_no_swapper_pki_tot,l3_misses_no_swapper_g,l3_misses_pki_no_swapper_g,l3_misses_no_swapper_p,l3_misses_pki_no_swapper_p,l3_misses_tot,l3_misses_pki_tot,l3_misses_g,l3_misses_pki_g,l3_misses_p,l3_misses_pki_p,fe_stalls_no_swapper_tot,fe_stalls_no_swapper_pki_tot,fe_stalls_no_swapper_g,fe_stalls_pki_no_swapper_g,fe_stalls_no_swapper_p,fe_stalls_pki_no_swapper_p,fe_stalls_tot,fe_stalls_pki_tot,fe_stalls_g,fe_stalls_pki_g,fe_stalls_p,fe_stalls_pki_p,llc_misses_tot,llc_misses_pki_tot,top5_insts,droidbg_insts,ddr_mbps,llcc_gold_mbps,llcc_prime_mbps,mig_intra_tot,mig_inter_gp,mig_intra_pkw,mig_inter_pkw_gp,preempt_cnt,preempt_pkw,wake_pc_tot_cnt,wake_pc_s_cnt,wake_pc_gp_cnt,wake_wfi_tot_cnt,wake_tot_cnt,freq_avg_g,freq_avg_p,l3_collapse_cnt,l3_collapse_dur,coloc_cnt,coloc_t,fboost_t,cboost_t,rboost_t,tot_boost_t,tot_boost_cnt,state1_time_residency,state_transition_cnt,active_tot,insts_tot_by_active_tot
KNP,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
GSVKFFC02,-23.5,-9.22271e+06,-1420,-23.5,-60.4,-116438,-135852,-0.9,-126923,-163283,-0.8,-99491.4,-78.4,-145144,-0.7,-27431.9,-21.6,-18138.9,-1.5,-470.4,-4,-388,-4.3,-82.4,-3.1,-479.2,-3.8,-395.1,-4,-84.1,-3.1,0,0,0,0,0,0,0,0,0,0,0,0,-41390.8,-355.5,-34536.6,-385.2,-6854.2,-255.9,-51000.8,-401.8,-43701.2,-439.2,-7299.6,-266.1,-2613.7,-20.6,-76637,-0.4,0,-15636.2,-5154.5,-107169,-2806,-197.1,-5.2,-23147,-42.6,-7985,0,-7985,-535751,-543735,-2257.1,-1106.7,0,0,0,0,0,0,0,0,0,-100,0,-80.2,-1583.6
GPHDR120,-8.1,-5.21223e+06,-980,-8.1,-0.3,-41780.7,-87308.7,0.3,-48004.1,-111144,0.3,-50547.1,-7.8,-112996,0.4,2543,7.8,1851.3,0,-284.6,-1.2,-292.9,-1.6,8.2,0.1,-287.5,-1.1,-295.3,-1.4,7.9,0.1,0,0,0,0,0,0,0,0,0,0,0,0,-38209.5,-144.7,-40146.7,-192.4,1937.2,33,-46363.9,-160.5,-48205,-205.8,1841,26.9,-3621.5,-19.1,-27862.7,0.6,3445.6,-4581.9,1039.8,76859,-4994,436.3,-5.1,19481,86,-2823,0,-2823,-267080,-269905,-795.9,-417,0,0,0,0,0,0,0,0,0,0,0,1.5,-326.4
HOK02W,2.2,-2.30385e+06,302,2.2,0.5,103694,13445.2,0.4,103212,452,0.5,-7792.7,-23.9,-45813.6,0.3,111005,23.9,46265.6,0.4,90.7,-0.5,-104.9,-0.5,195.6,-0.1,89.6,-0.4,-105.2,-0.5,194.8,-0.1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-5253.3,-21.7,-1527.7,nan,9011.5,-2193.7,4595.1,169171,5453,278.5,9.2,57618,94.4,-18088,0,-18088,-6271,-24358,-643.4,-400.3,0,0,0,0,0,0,0,0,0,0,0,84.4,-195.5
SRVKGH02,-3.6,-789875,-219,-3.6,0,-28932.2,-26618,0.1,-30465,-35168.6,0,-18620.2,-1.8,-20357.8,0.1,-11844.8,1.8,-14810.8,0.1,-28.1,0.3,-36.2,0,8,0.4,-29.1,0.3,-35.9,0,6.8,0.4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-7944.3,-24.3,-27563.6,nan,10513.9,-1105.8,-518.2,-52113,11282,-48.4,21.4,-6633,5.4,-33302,0,-33302,-41815,-75113,10.5,-126.6,0,0,0,0,0,0,0,0,0,0,0,-29.4,81.2
WWVKCP01,-2.9,-2.27517e+06,-174,-2.9,0,-16862.4,-47775.7,0.2,-21189.9,-63702.9,0.2,-29191.2,-7.2,-74570.4,0.3,8001.3,7.2,10867.5,-0.2,-76.6,-0.1,-143.5,-0.4,66.9,1.1,-77.7,0,-144.6,-0.3,66.9,1,0,0,0,0,0,0,0,0,0,0,0,0,-17198,-70.1,-24916.1,-131.5,7718,134.9,-22342.6,-86.3,-30016.4,-147.6,7673.8,129.8,-3994.3,-21.7,-7723.3,nan,7452,-1100.6,522,42163,-4185,353.5,-7,29274,147.7,-28728,0,-28728,-180017,-208741,-828.1,-206.2,0,0,0,0,0,0,0,0,0,99.9,-2,25.7,-423.8
ZZZVKLS01,-4.9,-5.11270e+06,-367,-4.9,-1.3,-36370.3,-19400.9,0,-43379.4,-55601.6,0,43792.7,22.5,16251.2,0.1,-87172.1,-22.5,-71852.8,0.1,-6.1,0.3,299.3,0.6,-305.3,1.4,-23.4,0.3,286,0.6,-309.4,1.2,0,0,0,0,0,0,0,0,0,0,0,0,-6885.4,8.2,23466.4,37.8,-30351.8,-66.9,-16402.7,-11.1,15772.1,12.1,-32174.8,-61,-6616.2,-17.1,-31780.9,16.4,9732.5

## % Delta

kpi,duration,line_cnt,commit_cnt,render_t,fps,insts_no_swapper_tot,cycles_no_swapper_tot,ipc_no_swapper_tot,insts_tot,cycles_tot,ipc_tot,insts_g,wl_pct_g,cycles_g,ipc_g,insts_p,wl_pct_p,cycles_p,ipc_p,l2_misses_no_swapper_tot,l2_misses_no_swapper_pki_tot,l2_misses_no_swapper_g,l2_misses_pki_no_swapper_g,l2_misses_no_swapper_p,l2_misses_pki_no_swapper_p,l2_misses_tot,l2_misses_pki_tot,l2_misses_g,l2_misses_pki_g,l2_misses_p,l2_misses_pki_p,l3_misses_no_swapper_tot,l3_misses_no_swapper_pki_tot,l3_misses_no_swapper_g,l3_misses_pki_no_swapper_g,l3_misses_no_swapper_p,l3_misses_pki_no_swapper_p,l3_misses_tot,l3_misses_pki_tot,l3_misses_g,l3_misses_pki_g,l3_misses_p,l3_misses_pki_p,fe_stalls_no_swapper_tot,fe_stalls_no_swapper_pki_tot,fe_stalls_no_swapper_g,fe_stalls_pki_no_swapper_g,fe_stalls_no_swapper_p,fe_stalls_pki_no_swapper_p,fe_stalls_tot,fe_stalls_pki_tot,fe_stalls_g,fe_stalls_pki_g,fe_stalls_p,fe_stalls_pki_p,llc_misses_tot,llc_misses_pki_tot,top5_insts,droidbg_insts,ddr_mbps,llcc_gold_mbps,llcc_prime_mbps,mig_intra_tot,mig_inter_gp,mig_intra_pkw,mig_inter_pkw_gp,preempt_cnt,preempt_pkw,wake_pc_tot_cnt,wake_pc_s_cnt,wake_pc_gp_cnt,wake_wfi_tot_cnt,wake_tot_cnt,freq_avg_g,freq_avg_p,l3_collapse_cnt,l3_collapse_dur,coloc_cnt,coloc_t,fboost_t,cboost_t,rboost_t,tot_boost_t,tot_boost_cnt,state1_time_residency,state_transition_cnt,active_tot,insts_tot_by_active_tot
KNP,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
GSVKFFC02,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,nan,-100,-100,-100,-100,-100,-100,-100,-100,-100,nan,-100,-100,-100,-100,-100,nan,nan,nan,nan,nan,nan,nan,nan,nan,-100,nan,-100,-100
GPHDR120,-13.5,-30.16,-13.69,-13.5,-0.25,-24.38,-43.94,33.33,-25.3,-46.72,37.5,-32.39,-9.48,-53.6,57.14,7.56,44.07,6.83,0,-56.82,-41.38,-65.7,-50,14.86,5.88,-56.01,-40.74,-64.74,-48.28,13.84,5.88,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,-47.88,-31.07,-58.19,-38.65,17.93,10,-49.7,-32.64,-58.94,-39.27,15.99,7.86,-100,-100,-23.38,600,inf,-45.6,52.97,17.64,-72.67,78.93,-58.62,26.6,92.57,-15.6,nan,-15.6,-34.69,-34.25,-50.09,-31.84,nan,nan,nan,nan,nan,nan,nan,nan,nan,0,nan,0.99,-26.01
HOK02W,3.81,-15.21,4.34,3.81,0.42,45.45,7.51,30.77,42.69,0.22,41.67,-4.46,-33.06,-28.04,27.27,165.68,86.28,109.27,25,16.46,-20.83,-24.6,-19.23,157.11,-5.26,15.89,-17.39,-24.05,-20,153.87,-5.26,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,-100,-100,-0.8,nan,inf,-16.74,98.42,117.34,63.84,125.79,70.23,143.07,152.75,-81.08,nan,-81.08,-1,-3.74,-43.66,-27.42,nan,nan,nan,nan,nan,nan,nan,nan,nan,0,nan,60.76,-11.23
SRVKGH02,-6.23,-6.35,-6.36,-6.23,0,-9.2,-11.21,7.69,-9.31,-13.58,0,-13.08,-4.14,-18.73,7.69,-6.41,3.19,-9.86,8.33,-2.22,7.5,-13.04,0,0.81,7.41,-2.27,7.69,-12.48,0,0.68,7.41,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,-100,-100,-11.79,nan,inf,-12.73,-3.01,-24.53,362.18,-14.14,428,-8.39,4.24,-73.65,nan,-73.65,-7.27,-12.1,1.3,-6.67,nan,nan,nan,nan,nan,nan,nan,nan,nan,0,nan,-13.86,5.26
WWVKCP01,-6.29,-22.26,-6.36,-6.29,0,-9.74,-25.97,22.22,-11.53,-30.68,22.22,-19.37,-8.78,-40.55,37.5,24.24,40,45.78,-14.29,-12.14,-2.78,-26.91,-10.53,68.55,36.67,-12.14,0,-26.72,-8.33,67.71,33.33,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,-26.82,-18.93,-45.43,-33.75,83.24,47,-31.01,-22.01,-48.13,-35.68,79.23,44.22,-100,-100,-6.91,nan,inf,-8.84,10.35,26.38,-82.95,113.3,-71.43,76.63,197.99,-83.65,nan,-83.65,-37.67,-40.75,-45.68,-16.51,nan,nan,nan,nan,nan,nan,nan,nan,nan,99900,-100,21.72,-27.29
ZZZVKLS01,-8.29,-21.08,-10.29,-8.29,-2.16,-10.14,-6.52,0,-11.24,-15.61,0,14.78,29.3,5.75,10,-97.25,-96.98,-97.39,8.33,-0.67,12,50.09,27.27,-96.46,38.89,-2.5,12.5,46.43,28.57,-96.3,33.33,nan,nan,nan,nan,nan,

# Top APP KPIs

In [71]:
def get_top_app_kpis(trace, trace_name):
    top_app_kpi_df = pd.DataFrame()
    stu = trace.df_events("sched_task_util").copy()
    stu = stu[stu.is_rtg == 1]
    top_app_pids = list(stu.pid.unique())
    sched_ctrs_df = trace.df_events("sched_switch_with_ctrs").copy()
    filtered_df = sched_ctrs_df[sched_ctrs_df['prev_pid'].isin(top_app_pids)].copy()
    
    # Ensure columns are numeric
    filtered_df['INST'] = pd.to_numeric(filtered_df['INST'], errors='coerce')
    filtered_df['CYC'] = pd.to_numeric(filtered_df['CYC'], errors='coerce')
    
    inst = filtered_df['INST'].sum() / 1000000
    cyc = filtered_df['CYC'].sum() / 1000000
    ipc = filtered_df['INST'].sum() / filtered_df['CYC'].sum()
    
    top_app_kpi_df['top_app_inst'] = [inst]
    top_app_kpi_df['top_app_cyc'] = [cyc]
    top_app_kpi_df['top_app_ipc'] = [ipc]
    top_app_kpi_df.index = [trace_name]
    
    return top_app_kpi_df


In [73]:
output_dict = {}

for build, traces in builds_dict.items():
    combined_top_app_kpi_df = pd.DataFrame()
    for trace in traces:
        combined_top_app_kpi_df = pd.concat([combined_top_app_kpi_df, get_top_app_kpis(trace, trace.name)], axis = 0)
    combined_top_app_kpi_df = combined_top_app_kpi_df.reindex(index=usecasestestplans_dict['GameDB'].keys(),  fill_value=0)
#     combined_top_app_kpi_df.index = combined_top_app_kpi_df.index.map(usecasestestplans_dict['GDoU_5.2'])
    combined_top_app_kpi_df.index.name = build
    combined_top_app_kpi_df = combined_top_app_kpi_df.round(2)
    output_dict[build] = combined_top_app_kpi_df
output_dict['KNP - Pakala'] = output_dict['KNP'] - output_dict['Pakala']
output_dict['% Delta'] = (output_dict['KNP - Pakala']/output_dict['Pakala']) * 100
print_dict_table(output_dict)

def export_to_excel(output_dict, filename='Top_APP_KPIs.xlsx'):
    with pd.ExcelWriter(filename) as writer:
        for build, output in output_dict.items():
            output.to_excel(writer, sheet_name=build, index=True)
export_to_excel(output_dict)

## KNP

,top_app_inst,top_app_cyc,top_app_ipc
KNP,,,
GSVKFFC02,0,0,0
GPHDR120,108765,83738.2,1.3
HOK02W,213321,118606,1.8
SRVKGH02,261861,190373,1.38
WWVKCP01,138931,115931,1.2
ZZZVKLS01,204.34,438.22,0.47


## Pakala

,top_app_inst,top_app_cyc,top_app_ipc
Pakala,,,
GSVKFFC02,106303,118163,0.9
GPHDR120,145890,154831,0.94
HOK02W,203755,149598,1.36
SRVKGH02,292080,214973,1.36
WWVKCP01,153296,156993,0.98
ZZZVKLS01,338155,266417,1.27


## KNP - Pakala

,top_app_inst,top_app_cyc,top_app_ipc
KNP,,,
GSVKFFC02,-106303,-118163,-0.9
GPHDR120,-37125,-71093.1,0.36
HOK02W,9565.62,-30991.6,0.44
SRVKGH02,-30219.3,-24600.5,0.02
WWVKCP01,-14364.9,-41061.8,0.22
ZZZVKLS01,-337950,-265978,-0.8


## % Delta

,top_app_inst,top_app_cyc,top_app_ipc
KNP,,,
GSVKFFC02,-100,-100,-100
GPHDR120,-25.4472,-45.9165,38.2979
HOK02W,4.69466,-20.7166,32.3529
SRVKGH02,-10.3462,-11.4435,1.47059
WWVKCP01,-9.37074,-26.1552,22.449
ZZZVKLS01,-99.9396,-99.8355,-62.9921


# Cluster-Sum Frequency Residency

In [75]:
display(Markdown("# Cluster-Sum Frequency Residency"))

cluster_sum_combined_dict = {}
for build, traces in builds_dict.items():  
    clusters = traces[0].plat_info['clusters'].keys()
    tmp_dict = {}
    for cluster in clusters:
        combined_cluster_sum_freq_df = pd.DataFrame()
        for i in range(len(traces)):
            df = traces[i].analysis.frequency.compare_cluster_freq_per_core_state_resid_combined([], cluster=cluster, normalize=True, norm_to_abs_time=True, rounding=2).filter(like = 'cl_avg active resid')
            df.index.name = cluster.upper()
            combined_cluster_sum_freq_df = pd.concat([combined_cluster_sum_freq_df, df], axis = 1)
        combined_cluster_sum_freq_df = combined_cluster_sum_freq_df.fillna(0)
        combined_cluster_sum_freq_df = combined_cluster_sum_freq_df.T
        combined_cluster_sum_freq_df.index = pd.Index([st.replace("cl_avg active resid ", "") for st in combined_cluster_sum_freq_df.index], name='cl_avg active resid')
        combined_cluster_sum_freq_df.index = combined_cluster_sum_freq_df.index.map(lambda x: get_usecase_name(x, usecasestestplans_dict))
        combined_cluster_sum_freq_df = combined_cluster_sum_freq_df.reindex(index=usecasestestplans_dict['GameDB'].keys(), fill_value=0)
        tmp_dict[cluster] = combined_cluster_sum_freq_df
    cluster_sum_combined_dict[build] = tmp_dict

common_clusters = [cluster for cluster in cluster_sum_combined_dict['KNP'].keys() if cluster in cluster_sum_combined_dict['Pakala'].keys()]
cluster_sum_combined_dict['Delta'] = {}
for cluster in common_clusters:
    cluster_sum_combined_dict['KNP_Alligned'] = cluster_sum_combined_dict['KNP'][cluster].copy()
    cluster_sum_combined_dict['Pakala_Alligned'] = cluster_sum_combined_dict['Pakala'][cluster].copy()
    cluster_sum_combined_dict['Pakala_Alligned'] = cluster_sum_combined_dict['Pakala_Alligned'].reindex(columns=cluster_sum_combined_dict['Pakala_Alligned'].columns.union(cluster_sum_combined_dict['KNP_Alligned'].columns, sort=False), fill_value=0)
    cluster_sum_combined_dict['KNP_Alligned'] = cluster_sum_combined_dict['KNP_Alligned'].reindex(columns=cluster_sum_combined_dict['KNP_Alligned'].columns.union(cluster_sum_combined_dict['Pakala_Alligned'].columns, sort=False), fill_value=0)
    cluster_sum_combined_dict['Delta'][cluster] = cluster_sum_combined_dict['KNP_Alligned'] - cluster_sum_combined_dict['Pakala_Alligned']
    for key in ['KNP_Alligned', 'Pakala_Alligned']:
        cluster_sum_combined_dict.pop(key, None)
        
for build, clusters in cluster_sum_combined_dict.items():
    display(Markdown("## {}".format(build)))
    print_dict_table_with_pallete(clusters)

with pd.ExcelWriter('Exit_Review_Cluster_Sum_Freq.xlsx') as writer:
    for build, clusters in cluster_sum_combined_dict.items():
            for cluster, table in clusters.items():
                table.to_excel(writer, sheet_name = build+'_'+cluster, index = True)

# Cluster-Sum Frequency Residency

## KNP

## Gold

GOLD,0,787,883,998,1113,1228,1324,1440,1555,1670,1785,1900,1996,2112,2227,2361,2496,2611,2745,2899,3033,3187,3302,3398,3513,3628
cl_avg active resid,,,,,,,,,,,,,,,,,,,,,,,,,,
GSVKFFC02,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
GPHDR120,61,37.38,0.96,0.58,0.04,0,0,0,0,0,0.01,0.01,0,0,0,0,0,0,0,0,0,0,0,0,0,0
HOK02W,61.43,35.08,0.47,0.32,0.41,0.76,0.47,0.14,0.12,0.25,0.13,0.05,0.06,0.06,0.04,0.09,0.03,0.01,0.02,0.01,0.01,0.01,0.01,0.01,0.01,0.01
SRVKGH02,67.2,22.54,10.25,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
WWVKCP01,57.1,6.52,16.98,9.83,4.89,1.9,1.59,0.29,0.16,0.24,0.21,0.15,0.02,0.04,0.02,0.01,0.01,0.01,0,0,0,0,0,0,0.01,0.01
ZZZVKLS01,53.87,0,0,0,0,0,0.07,0.18,0.9,2.91,7.72,10.36,7.96,7.49,4.98,2.58,0.89,0.11,0,0,0,0,0,0,0,0


## Prime

...

## Delta

## Gold

GOLD,0,556,748,787,883,960,998,1113,1152,1228,1324,1363,1440,1555,1670,1785,1900,1996,2112,2227,2361,2400,2496,2611,2745,2899,2918,3033,3072,3187,3302,3321,3398,3513,3532,3628
cl_avg active resid,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
GSVKFFC02,-54.71,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-4.64,0,-28.1,0,-11.79,0,0,-0.75,0,0,0,0,0,0,0,0,0,0,0
GPHDR120,-2.27,0,-0.02,37.38,0.96,-0.19,0.58,0.04,-1.74,0,0,-8.15,0,-14.19,0,-10.54,0.01,-1.34,0,-0.24,0,-0.06,0,0,-0.1,0,-0.01,0,-0.05,0,0,-0.01,0,0,-0.07,0
HOK02W,-6.46,0,-0.1,35.08,0.47,-2.82,0.32,0.41,-5.86,0.76,0.47,-11.68,0.14,-6.46,0.25,-1.4,0.05,-1.18,0.06,-1.25,0.09,-0.12,0.03,0.01,-0.19,0.01,-0.11,0.01,-0.09,0.01,0.01,-0.08,0.01,0.01,-0.4,0.01
SRVKGH02,5.61,-6.85,-17,22.54,10.25,-14.26,0,0,-0.1,0,0,-0.18,0,-0.02,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
WWVKCP01,-6.46,0,-0.04,6.52,16.98,-0.12,9.83,4.89,-0.15,1.9,1.59,-1.76,0.29,-10.79,0.24,-13.07,0.15,-4.69,0.04,-3.32,0.01,-1.45,0.01,0.01,-0.23,0,-0.05,0,-0.05,0,0,-0.03,0,0.01,-0.28,0.01
ZZZVKLS01,-4.08,0,0,0,0,0,0,0,-0.36,0,0.07,-2.66,0.18,-6.12,2.91,-6.13,10.36,-1.25,7.49,0.03,2.58,-2.56,0.89,0.11,-1.36,0,-0.07,0,0,0,0,0,0,0,0,0


## Prime

PRIME,0,883,1017,1132,1209,1267,1382,1401,1497,1632,1689,1747,1862,1958,1977,2112,2227,2246,2438,2649,2668,2841,2880,3072,3129,3283,3398,3513,3648,3763,3801,3859,3974,4070,4089,4185,4204,4320,4396,4531,4608,4742
cl_avg active resid,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
GSVKFFC02,-65.54,0,-28.56,0,-3.44,0,0,-0.76,0,0,-0.23,0,0,-0.17,0,0,0,-0.54,-0.67,-0.02,0,-0.07,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
GPHDR120,-14.23,28.95,-6.76,0.15,-1.34,0.23,0.07,-1.7,0.03,0.04,-2.91,0.01,0.04,-1.67,0.01,0.02,0,-0.8,-0.09,-0.01,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-0.02,-0.04,0,0,0,0
HOK02W,-46.5,43.37,5.44,4.71,-4.3,4.05,1.8,-6.26,0.55,1.31,-4.96,1.2,0.53,-1.05,0.49,0.64,0.32,-1.29,0.37,-0.15,0.12,-0.09,0.07,-0.18,0.1,-0.11,0.07,-0.06,0.04,0,-0.06,0,0.02,0,-0.05,0.01,-0.05,-0.12,0,0,0,0
SRVKGH02,-2.09,0.01,0.01,0.39,-0.06,1.6,3.28,-0.91,10.72,16.71,-24.91,12.21,11.48,-33.89,5.58,3.26,1.88,-7.12,0.27,-0.21,0.74,-0.23,0.37,-0.1,0.46,-0.03,0.18,-0.07,0.22,0.11,-0.04,0.03,0.03,0.02,0,0.02,0,-0.01,0.01,0,0,0.07
WWVKCP01,-18.95,22.79,-4.41,2.81,-2.58,1.43,0.48,-0.84,0.45,0.28,-0.53,0.15,0.57,-1.3,0.06,0.52,0.06,-1.2,0.34,-0.05,0.07,-0.02,0.04,-0.02,0.03,-0.01,0.02,-0.01,0.02,0.06,-0.02,0,0,0,-0.02,0.01,-0.01,-0.25,0,0,0,0.01
ZZZVKLS01,31.46,0.03,1.08,0.14,-2.1,0,0,-1.77,0,0,-11.86,0,0,-8.41,0,0,0,-6.82,-1.22,-0.25,0,-0.29,0,-0.01,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


# DDR APPS Vote Residency

In [76]:
combined_ddr_res_dict = {}
for build, traces in builds_dict.items(): 
    combined_ddr_res_df = pd.DataFrame()
    for trace in traces:
        dfa = trace.analysis.bus.compare_ddr_vote_summary([])
        dfa.index = sorted(dfa.index.astype(int))
        dfa = dfa.fillna(0)
        combined_ddr_res_df = pd.concat([combined_ddr_res_df, dfa], axis = 1)
    combined_ddr_res_df = combined_ddr_res_df.T
    combined_ddr_res_df.index.name = 'DDR APSS Vote Residency '+ build
    combined_ddr_res_df = combined_ddr_res_df.reindex(index=usecasestestplans_dict['GameDB'].keys(), fill_value=0)
#     combined_ddr_res_df.index = combined_ddr_res_df.index.map(usecasestestplans_dict)
    combined_ddr_res_dict[build] = combined_ddr_res_df
    

combined_ddr_res_dict['KNP_Alligned'] = combined_ddr_res_dict['KNP'].copy()
combined_ddr_res_dict['Pakala_Alligned'] = combined_ddr_res_dict['Pakala'].copy()


combined_ddr_res_dict['KNP_Alligned'] = combined_ddr_res_dict['KNP_Alligned'].reindex(columns=combined_ddr_res_dict['KNP_Alligned'].columns.union(combined_ddr_res_dict['Pakala_Alligned'].columns, sort=False), fill_value=0)
combined_ddr_res_dict['Pakala_Alligned'] = combined_ddr_res_dict['Pakala_Alligned'].reindex(columns=combined_ddr_res_dict['Pakala_Alligned'].columns.union(combined_ddr_res_dict['KNP'].columns, sort=False), fill_value=0)

combined_ddr_res_dict['KNP - Pakala'] = combined_ddr_res_dict['KNP_Alligned'] - combined_ddr_res_dict['Pakala_Alligned']

for key in ['KNP_Alligned', 'Pakala_Alligned']:
    combined_ddr_res_dict.pop(key, None)
display(Markdown("# DDR APSS Vote Residency"))
for build, table in combined_ddr_res_dict.items():
#     display(Markdown("### {}".format(build)))
    styled_df = table.style.background_gradient(cmap='Pastel2', axis = 1)
    display(HTML(styled_df.render()))



# DDR APSS Vote Residency

,0,547,1353,1555,1708,2092,2736,3187,3686,4224,4761,5333
DDR APSS Vote Residency KNP,,,,,,,,,,,,
GSVKFFC02,0,0,0,0,0,0,0,0,0,0,0,0
GPHDR120,0,42.44,45.46,10.46,0.01,0.68,0.07,0.87,0,0,0,0
HOK02W,0,0,0.15,1.1,0.12,4.93,0.03,93.67,0,0,0,0
SRVKGH02,0,0,0.06,0,0,0,0,99.94,0,0,0,0
WWVKCP01,0,0.01,0.3,36.84,0.63,15.86,0.21,46.15,0,0,0,0
ZZZVKLS01,0,0,0.22,0.73,0.05,4.42,0,94.59,0,0,0,0


,0,200,547,1353,1555,1708,2092,2736,3187,3686,4224,4761
DDR APSS Vote Residency Pakala,,,,,,,,,,,,
GSVKFFC02,0,0,0,0,0,0.02,0,0,99.05,0.78,0.15,0
GPHDR120,0,0,0.02,0.67,11.39,0,25.17,0,62.24,0.44,0.04,0.03
HOK02W,0,0,0.24,3,7.74,0,5.24,0,80.55,2.85,0.28,0.1
SRVKGH02,0,0,0,0,0,0.05,0,0,93.93,6.02,0,0
WWVKCP01,0,0,0,0,0.47,0,3.81,0,93.69,1.75,0.07,0.21
ZZZVKLS01,0,0,0,0,0,0,0.17,0,94.83,4.79,0.01,0.2


,0,200,547,1353,1555,1708,2092,2736,3187,3686,4224,4761,5333
DDR APSS Vote Residency KNP,,,,,,,,,,,,,
GSVKFFC02,0,0,0,0,0,-0.02,0,0,-99.05,-0.78,-0.15,0,0
GPHDR120,0,0,42.42,44.79,-0.93,0.01,-24.49,0.07,-61.37,-0.44,-0.04,-0.03,0
HOK02W,0,0,-0.24,-2.85,-6.64,0.12,-0.31,0.03,13.12,-2.85,-0.28,-0.1,0
SRVKGH02,0,0,0,0.06,0,-0.05,0,0,6.01,-6.02,0,0,0
WWVKCP01,0,0,0.01,0.3,36.37,0.63,12.05,0.21,-47.54,-1.75,-0.07,-0.21,0
ZZZVKLS01,0,0,0,0.22,0.73,0.05,4.25,0,-0.24,-4.79,-0.01,-0.2,0


# LLCC APPS Vote Residency

In [77]:
combined_llcc_res_dict = {}
for build, traces in builds_dict.items(): 
    combined_llcc_res_df = pd.DataFrame()
    for trace in traces:
        dfa = trace.analysis.bus.compare_llcc_vote_summary([])
        dfa.index = sorted(dfa.index.astype(int))
        dfa = dfa.fillna(0)
        combined_llcc_res_df = pd.concat([combined_llcc_res_df, dfa], axis = 1)
    combined_llcc_res_df = combined_llcc_res_df.T
    combined_llcc_res_df.index.name = 'LLCC APPS Vote Residency'
    combined_llcc_res_df = combined_llcc_res_df.reindex(index=usecasestestplans_dict['GameDB'].keys(), fill_value=0)
#     combined_llcc_res_df.index = combined_llcc_res_df.index.map(usecase_name_dict)
    combined_llcc_res_dict[build] = combined_llcc_res_df

combined_llcc_res_dict['KNP_Alligned'] = combined_llcc_res_dict['KNP'].copy()
combined_llcc_res_dict['Pakala_Alligned'] = combined_llcc_res_dict['Pakala'].copy()


combined_llcc_res_dict['KNP_Alligned'] = combined_llcc_res_dict['KNP_Alligned'].reindex(columns=combined_llcc_res_dict['KNP_Alligned'].columns.union(combined_llcc_res_dict['Pakala'].columns, sort=False), fill_value=0)
combined_llcc_res_dict['Pakala_Alligned'] = combined_llcc_res_dict['Pakala_Alligned'].reindex(columns=combined_llcc_res_dict['Pakala_Alligned'].columns.union(combined_llcc_res_dict['KNP'].columns, sort=False), fill_value=0)


combined_llcc_res_dict['KNP - Pakala'] = combined_llcc_res_dict['KNP_Alligned'] - combined_llcc_res_dict['Pakala_Alligned']

for key in ['KNP_Alligned', 'Pakala_Alligned']:
    combined_llcc_res_dict.pop(key, None)
# combined_llcc_res_dict['% Delta'] = (combined_llcc_res_dict['Pakala - Lanai']/combined_llcc_res_dict['Lanai_Alligned']) * 100
display(Markdown("# LLCC APPS Vote Residency"))
print_dict_table_with_pallete(combined_llcc_res_dict)

with pd.ExcelWriter('DDR_LCC_APPS_Vote_Resy.xlsx') as writer:
    for build, table in combined_ddr_res_dict.items():
        table.to_excel(writer, sheet_name = build+'_'+'DDR_Res', index = True)
    for build, table in combined_llcc_res_dict.items():
        table.to_excel(writer, sheet_name = build+'_'+'LLCC_Res', index = True)

# LLCC APPS Vote Residency

## KNP

,0,282,350,533,605,806,933,1066,1211,1350
LLCC APPS Vote Residency,,,,,,,,,,
GSVKFFC02,0,0,0,0,0,0,0,0,0,0
GPHDR120,0,53.4,34.37,10.61,0.6,1.02,0,0,0,0
HOK02W,0,0,0,1.26,4.93,93.81,0,0,0,0
SRVKGH02,0,0,0,0,0.01,99.71,0.28,0,0,0
WWVKCP01,0,0.01,0.04,38.11,15.48,46.24,0.12,0,0,0
ZZZVKLS01,0,0.02,0,0.81,4.43,94.7,0.03,0,0,0


## Pakala

,0,350,533,600,806,933,1066,1211
LLCC APPS Vote Residency,,,,,,,,
GSVKFFC02,0,0,0,0.02,98.86,0.27,0.7,0.15
GPHDR120,0,0.84,15.87,20.87,62.29,0.01,0.12,0
HOK02W,0,3.24,7.8,5.19,80.85,2.35,0.48,0.1
SRVKGH02,0,0,0,0.05,97.98,0.7,0.37,0.9
WWVKCP01,0,0,0.78,3.47,94.58,0.78,0.13,0.26
ZZZVKLS01,0,0,0,0.2,98.76,0.71,0.07,0.27


## KNP - Pakala

,0,282,350,533,600,605,806,933,1066,1211,1350
LLCC APPS Vote Residency,,,,,,,,,,,
GSVKFFC02,0,0,0,0,-0.02,0,-98.86,-0.27,-0.7,-0.15,0
GPHDR120,0,53.4,33.53,-5.26,-20.87,0.6,-61.27,-0.01,-0.12,0,0
HOK02W,0,0,-3.24,-6.54,-5.19,4.93,12.96,-2.35,-0.48,-0.1,0
SRVKGH02,0,0,0,0,-0.05,0.01,1.73,-0.42,-0.37,-0.9,0
WWVKCP01,0,0.01,0.04,37.33,-3.47,15.48,-48.34,-0.66,-0.13,-0.26,0
ZZZVKLS01,0,0.02,0,0.81,-0.2,4.43,-4.06,-0.68,-0.07,-0.27,0


In [87]:
def top_threads_per_case_single(trace_list, metric_col='total_insts_mill', top_n=5):
  
    
    traces_by_name = {t.name: t for t in trace_list}
    rows = {}

    for usecase in usecasestestplans_dict['GameDB'].keys():
        
        trace = traces_by_name.get(usecase)
        if trace is None:
            matches = [t for t in trace_list if usecase in t.name]
            trace = matches[0] if matches else None
        if trace is None:
            continue

        
        try:
            df = trace.analysis.interface_all_kpis.tasks_compare_interleave_task_stats_detailed_with([]).round(2)
        except Exception as e:
            print(f"[TopThreadsSingle] 跳过 usecase={usecase}，详细统计失败：{e}")
            continue

        
        if metric_col in df.columns:
            series = df[metric_col]
        else:
            filtered = df.filter(like=metric_col)
            if filtered.shape[1] == 0:
                print(f"[TopThreadsSingle] usecase={usecase} 未找到指标列：{metric_col}")
                continue
            series = filtered.iloc[:, 0]

        # name priority：thread > task > combined key
        if 'thread' in df.columns:
            series = series.copy()
            series.index = df['thread']
        elif 'task' in df.columns:
            series = series.copy()
            series.index = df['task']
        else:
            cols_comb = [c for c in ['pid', 'task', 'config'] if c in df.columns]
            if cols_comb:
                combined = df[cols_comb].apply(lambda r: '_'.join(r.values.astype(str)), axis=1)
                series = series.copy()
                series.index = combined

        # TopN
        series = series.fillna(0)
        top_series = series.sort_values(ascending=False).head(top_n)

        row = {}
        for i, (name, val) in enumerate(top_series.items(), start=1):
            row[f'thread_{i}'] = str(name)
            row[f'value_{i}'] = round(float(val), 2)
        rows[usecase] = row

    out_df = pd.DataFrame.from_dict(rows, orient='index')
    out_df.index.name = f'Top{top_n} Threads by {metric_col}'
    return out_df


In [92]:
display(Markdown("### Top 5 Threads per Case - KNP"))
knp_top_threads_df = top_threads_per_case_single(trace_list_knp, metric_col='total_insts_mill', top_n=5)
display(HTML(knp_top_threads_df.style.background_gradient(cmap='Pastel2', axis = 1).render()))

display(Markdown("### Top 5 Threads per Case - Pakala"))
pakala_top_threads_df = top_threads_per_case_single(trace_list_pakala, metric_col='total_insts_mill', top_n=5)
display(HTML(pakala_top_threads_df.style.background_gradient(cmap='Pastel2', axis = 1).render()))

with pd.ExcelWriter('Exit_Review_Top_Threads.xlsx') as writer:
    knp_top_threads_df.to_excel(writer, sheet_name='Top5Threads_KNP', index=True)
    pakala_top_threads_df.to_excel(writer, sheet_name='Top5Threads_PAKALA', index=True)


### Top 5 Threads per Case - KNP

,thread_1,value_1,thread_2,value_2,thread_3,value_3,thread_4,value_4,thread_5,value_5
Top5 Threads by total_insts_mill,,,,,,,,,,
GPHDR120,Thread-,35706.4,Thread-,31526.3,RHIThread,28910.1,RenderThread,19900,TaskGraphNP,13145.7
HOK02W,UnityMain,86978.9,UnityMain,86855.4,Job.worker,50921.2,UnityGfxDeviceW,49384.4,Job.worker,28014.4
SRVKGH02,UnityGfxDeviceW,78431.1,UnityMain,75421,UnityMain,75311.8,Job.Worker,69725.4,NativeThread,17860.6
WWVKCP01,RenderThread,35489.7,GameThread,35154.5,GameThread,35143.7,RHIThread,20485.5,TaskGraphHP,17609.7
ZZZVKLS01,UnityMain,84302.5,UnityMain,84281.9,Job.Worker,82461.8,UnityGfxDeviceW,50925.1,Background Job.,29577.1


### Top 5 Threads per Case - Pakala

,thread_1,value_1,thread_2,value_2,thread_3,value_3,thread_4,value_4,thread_5,value_5
Top5 Threads by total_insts_mill,,,,,,,,,,
GPHDR120,Thread-,50702.9,Thread-,44200.4,RHIThread,33860.3,RenderThread,26253.9,TaskGraphNP,17888.6
HOK02W,UnityMain,76825.9,UnityMain,76187,Job.worker,55476.2,UnityGfxDeviceW,52023,Job.worker,30413
SRVKGH02,UnityGfxDeviceW,97334.7,UnityMain,83484.3,UnityMain,83381.3,Job.Worker,70097.5,NativeThread,18715.2
WWVKCP01,RenderThread,37085.4,GameThread,34712.8,GameThread,34703.7,RHIThread,25661.2,TaskGraphHP,17078.6
ZZZVKLS01,UnityMain,93585.1,UnityMain,93548.8,Job.Worker,91964.7,UnityGfxDeviceW,63936.1,Job.Worker,30920.3


In [ ]:
def task_stat_delta_at_dou(trace_list_1, trace_list_2, column_item):
    pakala_list = []
    lanai_list = []
    for trace in trace_list_1:
        pakala_list.append(trace.name)
    for trace in trace_list_2:
        lanai_list.append(trace.name)
    intersection_list = [item for item in pakala_list if item in lanai_list]

    common_traces_pakala = [trace for trace in trace_list_1 if trace.name in intersection_list]
    common_traces_lanai = [trace for trace in trace_list_2 if trace.name in intersection_list]
    combined_traces = common_traces_pakala + common_traces_lanai

    for trace in combined_traces:
        trace.name = trace.name + trace.plat_info['target_name']


    task_delta_df = pd.DataFrame()
    for usecase in usecasestestplans_dict['GameDB'].keys():
        if usecase in intersection_list:
            tmp_list = []
            for trace in combined_traces:
                if trace.name == usecase +  trace.plat_info['target_name'] :
                    tmp_list.append(trace)
                indices = [index for index, value in enumerate(combined_traces) if usecase in value.name ]
            
            df = combined_traces[indices[0]].analysis.interface_all_kpis.tasks_compare_delta_task_stats_with(combined_traces[indices[1]] ).round(2).filter(like=column_item).fillna(0)
            df.columns = [usecase]
            df = df.head(20)

            task_delta_df = pd.concat([task_delta_df, df], axis=1).fillna(0)

    task_delta_df['abs_sort_wtg'] = task_delta_df.abs().sum(axis=1)
    task_delta_df = task_delta_df.sort_values("abs_sort_wtg", ascending=False)
    task_delta_df.name = "Total Workload (d pct)"
    task_delta_df = task_delta_df.T
    task_delta_df = task_delta_df.reindex(index=list(usecasestestplans_dict['GameDB'].keys()) + ['abs_sort_wtg'], fill_value=0)
#     task_delta_df.index = task_delta_df.index.map(usecase_name_dict)
     

    for trace in trace_list_2:
        trace.name = trace.name.replace("sm8750", "")
    for trace in trace_list_1:
        trace.name = trace.name.replace("sm8850", "")
    task_delta_df.index.name = "total_insts_mill delta"
    task_delta_df.index = task_delta_df.index.fillna('abs_sort_wtg')
    return task_delta_df
#     display(HTML(task_delta_df.style.render())) 


In [73]:
display(Markdown("# Total_insts_mill delta"))
top_task_delta_df = task_stat_delta_at_dou(trace_list_knp, trace_list_pakala, 'total_insts_mill delta')
styled_df = top_task_delta_df.style.background_gradient(cmap='Pastel2', axis = 1)
display(HTML(styled_df.render()))
with pd.ExcelWriter('Exit_Review_Task_Stat_Delta.xlsx') as writer:
    top_task_delta_df.to_excel(writer, sheet_name = 'total_insts_mill delta', index = True)

# Total_insts_mill delta

GPHDR120
HOK02W
SRVKGH02
WWVKCP01
ZZZVKLS01
GPHDR120
HOK02W
SRVKGH02
WWVKCP01
ZZZVKLS01


,expr,UnityGfxDeviceW,UnityMain,Thread-,swapper,cnss_diag,binder:early_virtmgr,RHIThread,Job.Worker,RenderThread,raw_out_,NativeThread,writer,surfaceflinger,TaskGraphNP,Job.worker,sh,V DefaultWorke,Pool -,SRPSubmitThread,QFPS_Main_Timer,FAsyncLoading,AAudio_,rcuop,rcuog,binder:,TimerDispatch,FastMixer,rcu_preempt,kworker,CriManaDecodeTh,Loading.Preload,binder:vendor.qti.hardware.display.composer-service,HeapTaskDaemon,binder:com.tencent.tmgp.sgame,Background Job.,IOPool -,CoreThread,app,APM-TickThread,CRI MPV Worker,AudioTrack,TaskGraphHP,PoolThread,GameThread,CachedAppOptimi,highpool,Loading.AsyncRe,TaskGraphBP,eduling.default,PHR
total_insts_mill delta,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
GSVKFFC02,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
GPHDR120,0,0,0,-14996.5,-6223.39,-3295.93,373.63,-4950.27,0,-6353.93,1653.34,-2977.81,-1777.88,-341.22,-4742.88,0,0,0,0,0,509.71,-973.02,0,-386.87,-561.02,0,0,0,-364.94,-549.15,0,0,0,0,0,0,0,0,0,-556.29,0,0,0,-459.18,0,0,0,0,0,0,196.04
HOK02W,91289.2,-2638.58,10153.1,7286.04,0,0,546.39,0,0,0,1765.82,0,0,-3864.14,0,-4555,4300.27,0,0,0,570.01,0,-1445.6,1004.36,0,-1295.64,-1203.99,1183.91,0,0,0,0,-987.6,0,779.29,0,0,651.6,-559.83,0,0,538.2,0,0,0,0,0,0,0,0,0
SRVKGH02,0,-18903.6,-8063.23,0,-1532.76,-2683.59,4891.56,0,-372.05,0,1602.3,-854.62,-1769.06,-199.45,0,0,0,0,0,0,510.19,0,-482.76,0,-321.22,0,0,0,-272.4,577.99,-578.24,-285.01,0,0,0,0,0,0,0,0,-543.52,0,0,0,0,0,0,-283.01,0,-218.69,0
WWVKCP01,0,0,0,210.58,-4327.44,-4013.61,279.73,-5175.75,0,-1595.7,0,760.07,0,0,0,0,0,-3666.46,-3307.41,0,406.65,-1153.08,0,0,-306.92,0,0,0,-221.99,0,0,0,0,202.43,0,0,-744.81,0,0,0,0,0,531.15,0,441.68,357.27,331.48,0,-237.8,0,0
ZZZVKLS01,0,-13011.1,-9282.57,-1739.04,-7009.04,-958.86,4773.19,0,-9502.94,0,1592.77,-858.22,-1757.53,-415.65,0,0,0,0,0,-2635.2,515.73,0,0,-234.12,-389.72,0,0,0,-274.69,0,-533.47,-750.62,0,618.96,0,-773.93,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
abs_sort_wtg,91289.2,34553.2,27498.9,24232.1,19092.6,10952,10864.5,10126,9874.99,7949.63,6614.23,5450.72,5304.47,4820.46,4742.88,4555,4300.27,3666.46,3307.41,2635.2,2512.29,2126.1,1928.36,1625.35,1578.88,1295.64,1203.99,1183.91,1134.02,1127.14,1111.71,1035.63,987.6,821.39,779.29,773.93,744.81,651.6,559.83,556.29,543.52,538.2,531.15,459.18,441.68,357.27,331.48,283.01,237.8,218.69,196.04


In [80]:
display(Markdown("# Total_insts_mill delta pct @ Usecase"))
top_task_delta_df = task_stat_delta_at_dou(trace_list_knp, trace_list_pakala, 'total_insts_mill d percent')
styled_df = top_task_delta_df.style.background_gradient(cmap='Pastel2', axis = 1)
display(HTML(styled_df.render()))
with pd.ExcelWriter('Exit_Review_Task_Stat_Delta_Pct.xlsx') as writer:
    top_task_delta_df.to_excel(writer, sheet_name = 'total_insts_mill delta', index = True)

# Total_insts_mill delta pct @ Usecase

GPHDR120
HOK02W
SRVKGH02
WWVKCP01
ZZZVKLS01


,expr,Thread-,UnityGfxDeviceW,swapper,UnityMain,RHIThread,cnss_diag,RenderThread,binder:early_virtmgr,TaskGraphNP,NativeThread,Job.Worker,raw_out_,writer,V DefaultWorke,Pool -,surfaceflinger,FAsyncLoading,Job.worker,sh,QFPS_Main_Timer,rcuog,SRPSubmitThread,rcuop,kworker,AAudio_,rcu_preempt,IOPool -,APM-TickThread,binder:,CriManaDecodeTh,TimerDispatch,FastMixer,TaskGraphHP,PoolThread,Loading.Preload,HeapTaskDaemon,binder:vendor.qti.hardware.display.composer-service,GameThread,binder:com.tencent.tmgp.sgame,Background Job.,CachedAppOptimi,highpool,CoreThread,CRI MPV Worker,app,AudioTrack,TaskGraphBP,PHR,Loading.AsyncRe,eduling.default
total_insts_mill delta,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
GSVKFFC02,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
GPHDR120,0,-10.58,0,-4.39,0,-3.49,-2.33,-4.48,0.26,-3.35,-2.1,0,1.17,-1.25,0,0,-0.24,-0.69,0,0,0.36,-0.4,0,-0.27,-0.39,0,-0.26,0,-0.39,0,0,0,0,0,-0.32,0,0,0,0,0,0,0,0,0,0,0,0,0,0.14,0,0
HOK02W,26.46,2.11,-0.76,0,2.94,0,0,0,0.16,0,0,0,0.51,0,0,0,-1.12,0,-1.32,1.25,0.16,0,0,0.29,0,-0.42,0,0,0,-0.38,0,-0.35,0.34,0,0,0,0,-0.29,0,0.23,0,0,0,0.19,0,-0.16,0.16,0,0,0,0
SRVKGH02,0,0,-6.37,-0.52,-2.72,0,-0.9,0,1.65,0,-0.29,-0.12,0.54,-0.6,0,0,-0.07,0,0,0,0.17,-0.11,0,0,0.2,-0.16,-0.09,0,0,0,-0.2,0,0,0,0,-0.1,0,0,0,0,0,0,0,0,-0.18,0,0,0,0,-0.1,-0.07
WWVKCP01,0,0.13,0,-2.66,0,-3.18,-2.47,-0.98,0.17,0,0.47,0,0,0,-2.26,-2.04,0,-0.71,0,0,0.25,-0.19,0,0,0,0,-0.14,-0.46,0,0,0,0,0,0.33,0,0,0.12,0,0.27,0,0,0.22,0.2,0,0,0,0,-0.15,0,0,0
ZZZVKLS01,0,-0.51,-3.8,-2.05,-2.71,0,-0.28,0,1.39,0,-0.25,-2.77,0.46,-0.51,0,0,-0.12,0,0,0,0.15,-0.11,-0.77,-0.07,0,0,-0.08,0,0,0,-0.16,0,0,0,0,-0.22,0.18,0,0,0,-0.23,0,0,0,0,0,0,0,0,0,0
abs_sort_wtg,26.46,13.33,10.93,9.62,8.37,6.67,5.98,5.46,3.63,3.35,3.11,2.89,2.68,2.36,2.26,2.04,1.55,1.4,1.32,1.25,1.09,0.81,0.77,0.63,0.59,0.58,0.57,0.46,0.39,0.38,0.36,0.35,0.34,0.33,0.32,0.32,0.3,0.29,0.27,0.23,0.23,0.22,0.2,0.19,0.18,0.16,0.16,0.15,0.14,0.1,0.07


In [81]:
lpm_map_dict = {1:'C4', 0:'WFI(C2)', -1:'Active(C0)'}
lpm_cycle_pct_dict = {}
for build, traces in builds_dict.items():
    tmp_dict = {}
    for cluster, cpus in traces[0].plat_info['clusters'].items():
        per_usecase_df = pd.DataFrame()
        
        for trace in traces:
            sum_df = pd.DataFrame({'cycles': [0, 0, 0]}, index = [-1, 0, 1])
            for cpu in cpus:
                cpu_idle_df = trace.df_events('cpu_idle').copy()
                cpu_freq_df = trace.df_events('cpu_frequency').copy()
                combined_df = pd.concat([cpu_idle_df,cpu_freq_df], axis = 0)
                tmp_df = combined_df[(combined_df['cpu_id'] == cpu) | (combined_df['cpu'] == cpu)]
                tmp_df = tmp_df.sort_index() 
                tmp_df['frequency'] = tmp_df['frequency'].ffill()
                tmp_df['state'] = tmp_df['state'].ffill()
                filtered_df = tmp_df.reset_index()
                filtered_df['time_spent'] = filtered_df['Time'].diff().shift(-1)
                filtered_df['frequency'] = filtered_df['frequency'].fillna(cpu_freq_df.iloc[0, -2])
                filtered_df['cycles'] = filtered_df['time_spent'] * filtered_df['frequency'] * 1000
                filtered_df = filtered_df[[ 'state','time_spent', 'cycles']  ]
                grouped_df = filtered_df.groupby('state').sum()
                final_df = grouped_df[['time_spent', 'cycles']]
                sum_df = sum_df + final_df
                final_df = grouped_df.copy()
                final_df.loc['sum'] = final_df.sum()
                per_usecase_df[trace.name] = sum_df['cycles']

        
            def normalize_row(row):
                return (row / row.sum()) * 100

            df_normalized = per_usecase_df.apply(normalize_row, axis=0)
            df_normalized = df_normalized.T.round(1)
            df_normalized.index.name = cluster+ '_' +'Cycle Residency %'
            df_normalized = df_normalized.reindex(index=usecasestestplans_dict['GameDB'].keys(), fill_value=0)
#             df_normalized.index = df_normalized.index.map(usecasestestplans_dict)
            df_normalized = df_normalized.rename(columns = lpm_map_dict)
            tmp_dict[cluster] = df_normalized
        
#         display(HTML(df_normalized.round(1).style.render()))
    lpm_cycle_pct_dict[build] = tmp_dict

common_clusters = [cluster for cluster in lpm_cycle_pct_dict['KNP'].keys() if cluster in lpm_cycle_pct_dict['Pakala'].keys()]
lpm_cycle_pct_dict['Delta'] = {}
for cluster in common_clusters:
    lpm_cycle_pct_dict['Delta'][cluster] = lpm_cycle_pct_dict['KNP'][cluster] - lpm_cycle_pct_dict['Pakala'][cluster]
      
display(Markdown("# LPM Cycle Residency Percentage"))
for build, clusters in lpm_cycle_pct_dict.items():
    display(Markdown("# {}".format(build)))
    print_dict_table_with_pallete(clusters)
    
with pd.ExcelWriter('LPM Cycle Residency Percentage.xlsx') as writer:
    top_task_delta_df.to_excel(writer, sheet_name = 'total_insts_mill delta', index = True)

# LPM Cycle Residency Percentage

# KNP

## Gold

,Active(C0),WFI(C2),C4
Gold_Cycle Residency %,,,
GSVKFFC02,0,0,0
GPHDR120,41,59,nan
HOK02W,39.4,60.6,nan
SRVKGH02,34.7,65.3,nan
WWVKCP01,44.2,55.8,nan
ZZZVKLS01,48.5,51.5,nan


## Prime

...

# Delta

## Gold

,Active(C0),WFI(C2),C4
Gold_Cycle Residency %,,,
GSVKFFC02,-45.3,-50.2,-4.5
GPHDR120,2.5,-2.5,nan
HOK02W,6.8,-6.8,nan
SRVKGH02,-9.9,9.9,nan
WWVKCP01,2.7,-2.7,nan
ZZZVKLS01,6.6,-3.4,nan


## Prime

,Active(C0),WFI(C2),C4
Prime_Cycle Residency %,,,
GSVKFFC02,-34.5,-1.7,-63.8
GPHDR120,13.6,-1.4,-12.3
HOK02W,45.1,4.5,-49.5
SRVKGH02,2.1,-0.5,-1.6
WWVKCP01,18.1,6.2,-24.4
ZZZVKLS01,-31.2,-2.2,33.5
